In [15]:

# 载入库
import xlrd
import xlwt
from xlwt import Workbook
import numpy as np


In [16]:

# 载入原始Excel表
loc = ('test.xlsx')


In [17]:

# 打开表格及工作簿
wb = xlrd.open_workbook(loc)
sheet = wb.sheet_by_index(0)


In [18]:

# 行数
row = sheet.nrows

# 载入膜号列数
col_of_index = [2]


In [36]:

# 数据处理、转置、编组
d = []
index = -1
for j in col_of_index:
    for i in range(row):
        cur = sheet.cell_value(i, j)
        if type(cur) == float:
            rValue = sheet.cell_value(i, j + 5)
            if type(rValue) == float or type(rValue) == int:
                if index < 0 or d[index][0] != int(cur):
                    d.append((int(cur), [rValue]))
                    index = index + 1
                else:
                    d[index][1].append(rValue)
print(d)                    

[(1970, [0.0329, 0.0494, 0.0557, 0.0515, 0.0564, 0.0567]), (1941, [0.056, 0.0542]), (1969, [0.0633, 0.0554, 0.0521, 0.0582, 0.0745, 0.0671, 0.0652, 0.0656]), (1942, [0.0643]), (1958, [0.1394, 0.0675, 0.0677, 0.0715, 0.0713]), (1937, [0.0699, 0.0627, 0.0688]), (19343, [0.0537]), (1934, [0.1119, 0.0516, 0.059])]


In [20]:

# 重复膜号
error = 0
s = set()
wrong_pos = set()
for i in range(index + 1):
    if d[i][0] in s:
        wrong_pos.add(d[i][0])
    else:
        s.add(d[i][0])


In [21]:

# 将处理好的数据保存
# 创建工作簿
wb = Workbook()
sheet1 = wb.add_sheet('数据处理')
sheet2 = wb.add_sheet('出错数据')


In [22]:

# 标色出错膜号规则
styleWrongPos = xlwt.easyxf('pattern: pattern solid, fore_colour ocean_blue; font: bold on;')
styleMoreOrLess = xlwt.easyxf('pattern: pattern solid, fore_colour red; font: bold on;')


In [23]:

# 写入抬头
sheet1.write(0, 0, '膜号')
sheet1.write(0, 1, 'RI值')
sheet1.write(0, 2, 'RI值')
sheet1.write(0, 3, 'RI值')
sheet1.write(0, 4, 'RI值')
sheet1.write(0, 5, 'AVG')
sheet1.write(0, 6, 'CV')
sheet2.write(0, 0, '膜号')
sheet2.write(0, 1, 'RI值')


In [42]:

# 先进行错误膜号的修改
for i in range(index):
    if i > 0:        # 输出错号,修改录入出错的情况再进行录入
        a = len(d[i - 1][1])
        b = len(d[i][1])
        print(b,d[i][0])
        if a + b == 4:                      # 修改录入错误的膜号
            for k in range(b):
                d[i][1].append(d[i - 1][1][k])
                d[i - 1][1].remove(d[i - 1][1][k])
                d[i - 1][1].append(0)
                wrong_pos.add(d[i - 1][0])
        elif a > b and a + b == 8:          # 修改忘记改号的膜号
            for k in range(4,a):
                d[i][1].append(d[i - 1][1][k])
                d[i - 1][1].remove(d[i - 1][1][k])
        elif a == 8 and b == 1:             # 修改另一种忘记改号的情况
            for k in range(4,a):
                d[i][1].append(d[i - 1][1][k])
                d[i][1].remove(d[i][1][0])
print(d)

4 1941
8 1969
1 1942
4 1958
4 1937
1 19343
[(1970, [0.0329, 0.0494, 0.0557, 0.0515]), (1941, [0.056, 0.0542, 0.0564, 0.0567]), (1969, [0.0633, 0.0554, 0.0521, 0.0582, 0.0745, 0.0671, 0.0652, 0.0656]), (1942, [0.0656]), (1958, [0.1394, 0.0675, 0.0677, 0.0715]), (1937, [0.0699, 0.0627, 0.0688, 0.0713]), (19343, [0.0537]), (1934, [0.1119, 0.0516, 0.059])]


In [13]:
# 再进行膜号的分别输出
n = 1
m = 1
j = 0
for i in range(index + 1):
    if d[i][0] in wrong_pos:
        sheet2.write(n, 0, d[i][0])
        for j in range(len(d[i][1])):
            sheet2.write(n, j + 1, d[i][1][j], styleWrongPos)
        n = n + 1
    elif len(d[i][1]) != 4:
        sheet2.write(n, 0, d[i][0])
        for j in range(len(d[i][1])):
            sheet2.write(n, j + 1, d[i][1][j], styleMoreOrLess)
        n = n + 1
    else:
        sheet1.write(m, 0, d[i][0])
        for j in range(len(d[i][1])):
            sheet1.write(m, j + 1, d[i][1][j])
        tmp = [float(x) for x in d[i][1]]
        mean = np.mean(tmp)
        std = np.std(tmp)
        sheet1.write(m, j + 2, mean)
        sheet1.write(m, j + 3, std / mean)
        m = m + 1       
                
#导出工作簿
wb.save('数据处理.xls')